In [1]:
import spacy
from fastcoref import spacy_component

In [5]:
text = 'Alice goes down the rabbit hole. Where she would discover a new reality beyond her expectations.'

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("fastcoref")

01/08/2023 02:31:26 - INFO - 	 missing_keys: []
01/08/2023 02:31:26 - INFO - 	 unexpected_keys: []
01/08/2023 02:31:26 - INFO - 	 mismatched_keys: []
01/08/2023 02:31:26 - INFO - 	 error_msgs: []
01/08/2023 02:31:26 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M


In [7]:
doc = nlp(text)
doc._.coref_clusters

01/08/2023 02:31:34 - INFO - 	 Tokenize 1 inputs...


  0%|          | 0/1 [00:00<?, ?ba/s]

01/08/2023 02:31:35 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[[(0, 5), (39, 42), (79, 82)]]

In [17]:
from fastcoref import FCoref, LingMessCoref
import pandas as pd 

In [ ]:
model = LingMessCoref()
model = FCoref()

In [ ]:
preds = model.predict(
   texts=['We are so happy to see you using our coref package. This package is very fast!']
)

In [24]:
data_df = pd.read_csv('../resources/data/nytimes-articles-to-extract-sources.csv')

In [27]:
text = data_df.iloc[0]['text']

In [29]:
import pyperclip 
pyperclip.copy(text)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Try with Spacy and use tricks

In [102]:
filename = '../tasks/quote_detection/other_platforms/data/training_data.jsonl'
training_data = []
with open(filename) as f:
    for line in f:
        training_data.append(json.loads(line))

In [99]:
training_datum = training_data[0]
sents = list(map(lambda x: x['sent'], training_datum['data']))

# get sent charspans
sent_lens = list(map(len, sents))
sent_idxs = np.cumsum([0] + sent_lens)
sent_char_spans = list(zip(sent_idxs[:-1], sent_idxs[1:]))

# get clusters
text = ''.join(sents)
doc = nlp(text)
clusters = doc._.coref_clusters

# from clusters, get cluster mappers of heads -> spans
s2h_mapper, _ = get_cluster_mappers(doc, clusters, is_char_clusters=True)
s2h_to_replace = {k:v for k, v in s2h_mapper.items() if k != v}
s2h_to_replace = {k:s2h_to_replace[k] for k in sorted(s2h_to_replace, key=lambda x: x[0])}

# make sure we don't have nest corefs
old_s2h_len = len(s2h_to_replace)
while True:
    k = list(s2h_to_replace.keys())
    nested_coref = list(filter(lambda x: x[1][0] < x[0][1], zip(k[:-1], k[1:]))) # compare adjacent sorted spans to make sure the start-index of one isn't less than the end-index of another 
    to_remove = list(map(lambda x: min(x, key=lambda y: y[1] -y[0]), nested_coref)) # if it is, take the larger span
    s2h_to_replace = {k:v for k,v in s2h_to_replace.items() if k not in to_remove} # repeat until there are no more overlapping spans
    new_s2h_len = len(s2h_to_replace)
    if new_s2h_len == old_s2h_len:
        break
    else:
        old_s2h_len = new_s2h_len

# resolve the corefs
test_sents = copy(sents)
for k in list(reversed(s2h_to_replace)):
    v = s2h_to_replace[k]
    k_sent_idx, (k_sent_s, k_sent_e) = convert_to_sent_idx_and_sent_char_span(k, sent_bins=sent_char_spans)
    v_sent_idx, (v_sent_s, v_sent_e) = convert_to_sent_idx_and_sent_char_span(v, sent_bins=sent_char_spans)
    
    # determine what to replace
    to_replace = test_sents[v_sent_idx][v_sent_s: v_sent_e]
    final_token = doc.char_span(*k)[-1]
    if final_token.tag_ in ["PRP$", "POS"]:
        to_replace = to_replace + "'s"

    
    test_sents[k_sent_idx] = (
        test_sents[k_sent_idx][:k_sent_s] + to_replace + test_sents[k_sent_idx][k_sent_e:]
    )
    
assert len(test_sents) == len(sents)

# Maybe try redoing the sentence boundaries?

In [335]:
from jellyfish import jaro_distance, levenshtein_distance
from polyleven import levenshtein
import jsonlines
import spacy
from copy import copy
nlp = spacy.load('en_core_web_lg')

In [307]:
orig_data_df = pd.read_csv('../resources/data/nytimes-articles-to-extract-sources.csv')

In [308]:
with jsonlines.open('../tasks/all_annotated_sources.jsonl') as f:
    training_data = list(f)

In [329]:
old_sents = list(map(lambda x: x['sent'].strip(), training_data[0]))
old_text = ' '.join(sents)

In [330]:
orig_data_df['found'] = False

In [331]:
# jaro_test = orig_data_df['text'].apply(lambda x: jaro_distance(x, text))
# difflib_test = orig_data_df['text'].apply(lambda x: SequenceMatcher(x, text).ratio())
lev_test = orig_data_df['text'].apply(lambda x: levenshtein(x, old_text))

In [332]:
e_id, orig_text = orig_data_df.loc[lev_test.argmin()][['entry_id', 'text']]

In [334]:
old_sents[:2]

['BANGKOK', '—']

In [336]:
doc = nlp(orig_text)

In [337]:
new_sents = list(map(str, doc.sents))

In [ ]:
curr_new_sent_idx = 0
new_sents_copy = copy(new_sents)


for old_sent_idx, old_sent in enumerate(old_sents):
    # to match sentences, first 
    if len(old_sent) > 2:
        

In [339]:
old_sents[0]

'BANGKOK'

# Test Quote Detection Model Load

In [1]:
from transformers import AutoModel, AutoConfig

In [5]:
import sys
sys.path.insert(0, '../tasks/quote_detection/other_platforms/src/')

In [6]:
from sentence_model import SentenceClassificationModel

In [14]:
config = AutoConfig.from_pretrained('alex2awesome/source-exploration__quote-detection__sentence-roberta-base')
model = SentenceClassificationModel.from_pretrained('alex2awesome/source-exploration__quote-detection__sentence-roberta-base', config=config)

Downloading:   0%|          | 0.00/681 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [15]:
config

RobertaConfig {
  "_name_or_path": "alex2awesome/source-exploration__quote-detection__sentence-roberta-base",
  "architectures": [
    "SentenceClassificationModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}